# Part 1: Getting started

In [2]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2020.1/bin:' + os.environ['PATH']
tf.config.threading.set_inter_op_parallelism_threads(8)
os.environ['OMP_NUM_THREADS'] = '1'

import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1
from keras import optimizers

2023-01-05 13:37:19.681930: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 13:37:19.832071: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-05 13:37:19.838094: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/xilinx/xrt/lib:
2023-01-05 13:37:19.838113: I tensorflow/stream_executor/cuda/cudart_stub.c

## Fetch the white wine dataset

In [3]:
df = pd.read_csv('./dataset.csv', sep = ';')
print (np.shape(df))
df

(4898, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Y
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
X = df.drop('Y', axis = 1).values
y = df.Y
print(y)

0       6
1       6
2       6
3       6
4       6
       ..
4893    6
4894    5
4895    6
4896    7
4897    6
Name: Y, Length: 4898, dtype: int64


In [4]:
y[1:]

1       6
2       6
3       6
4       6
5       6
       ..
4893    6
4894    5
4895    6
4896    7
4897    6
Name: Y, Length: 4897, dtype: int64

In [5]:
le = LabelEncoder()
print(y)
y = le.fit_transform(y)
print(le.classes_)
y = to_categorical(y, 7)
print(y)

0       6
1       6
2       6
3       6
4       6
       ..
4893    6
4894    5
4895    6
4896    7
4897    6
Name: Y, Length: 4898, dtype: int64
[3 4 5 6 7 8 9]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
print(y[1])

[0. 0. 0. 1. 0. 0. 0.]


In [7]:
print(X)
print(y)

[[ 7.    0.27  0.36 ...  3.    0.45  8.8 ]
 [ 6.3   0.3   0.34 ...  3.3   0.49  9.5 ]
 [ 8.1   0.28  0.4  ...  3.26  0.44 10.1 ]
 ...
 [ 6.5   0.24  0.19 ...  2.99  0.46  9.4 ]
 [ 5.5   0.29  0.3  ...  3.34  0.38 12.8 ]
 [ 6.    0.21  0.38 ...  3.26  0.32 11.8 ]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [9]:
X_test

array([[ 6.        ,  0.29      ,  0.41      , ...,  3.09      ,
         0.59      , 10.96666667],
       [ 5.4       ,  0.53      ,  0.16      , ...,  3.2       ,
         0.53      , 13.2       ],
       [ 7.1       ,  0.25      ,  0.39      , ...,  3.28      ,
         0.43      , 12.2       ],
       ...,
       [ 7.        ,  0.4       ,  0.25      , ...,  3.        ,
         0.55      , 11.4       ],
       [ 6.8       ,  0.36      ,  0.24      , ...,  3.27      ,
         0.34      , 12.6       ],
       [ 6.3       ,  0.41      ,  0.3       , ...,  3.53      ,
         0.79      , 11.7       ]])

In [10]:
ls=np.argmax(Y_test,axis=1)+3
print(ls)
#np.savetxt('output_classes.dat',ls, fmt='%d')

[7 8 8 ... 6 7 7]


In [11]:
print(X_test)

[[ 6.          0.29        0.41       ...  3.09        0.59
  10.96666667]
 [ 5.4         0.53        0.16       ...  3.2         0.53
  13.2       ]
 [ 7.1         0.25        0.39       ...  3.28        0.43
  12.2       ]
 ...
 [ 7.          0.4         0.25       ...  3.          0.55
  11.4       ]
 [ 6.8         0.36        0.24       ...  3.27        0.34
  12.6       ]
 [ 6.3         0.41        0.3        ...  3.53        0.79
  11.7       ]]


In [12]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",Y_train.shape)
print("Shape of y_test",Y_test.shape)

Shape of X_train:  (3428, 11)
Shape of X_test:  (1470, 11)
Shape of y_train:  (3428, 7)
Shape of y_test (1470, 7)


In [13]:
print(X_test[1])

[5.4000e+00 5.3000e-01 1.6000e-01 2.7000e+00 3.6000e-02 3.4000e+01
 1.2800e+02 9.8856e-01 3.2000e+00 5.3000e-01 1.3200e+01]


In [14]:
import pickle
from joblib import load, dump
loaded_model = load('./WhiteWine.MLP_clf.joblib')

/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.24.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.24.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
w1=loaded_model.coefs_[0]
b1=loaded_model.intercepts_[0]
w2=loaded_model.coefs_[1]
b2=loaded_model.intercepts_[1]

wb1=[]
wb1.append(w1)
wb1.append(b1)

wb2=[]
wb2.append(w2)
wb2.append(b2)

In [16]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

import tensorflow as tf
import os
import pandas as pd
import seaborn as sb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights
from sklearn.preprocessing import MinMaxScaler
import hls4ml
import write_mlp_mergemult_ps as wv

/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


In [17]:
loaded_model

MLPClassifier(beta_1=0.4274478353845527, beta_2=0.729060367703723,
              epsilon=0.05766126202404963, hidden_layer_sizes=4,
              learning_rate='invscaling', max_iter=150,
              momentum=0.6907612738337751, nesterovs_momentum=False,
              solver='lbfgs', validation_fraction=0.21980439785647188)

### genetic algorithm to determine the relu size, weight size, bias size and sparsity
x1: relu_size
x2: weight_size
x3: bias_size
x4: sparsity

problem: max( blackbox(x1,x2,x3,x4)
         min ( x1*x2*x3)
       s.t
         2 < x1,x2,x3 < 8
         x4 belongs to [0.2, 0.3, 0.4, 0.5]

In [18]:
import blackbox as bb
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.problem import Problem
from pymoo.operators.repair.rounding import RoundingRepair
import area as ar
from importlib import reload
reload(bb)
reload(ar)

<module 'area' from '/home/argykokk/Desktop/TC/genetic_whitewine/area.py'>

In [19]:
class MyProblem(ElementwiseProblem):

    def __init__(self,weightsbiases1,weightsbiases2,layer1,layer2,layer3,X_test,X_train,Y_test,Y_train):
        self.weightsbiases1=weightsbiases1
        self.weightsbiases2=weightsbiases2
        self.layer1=layer1
        self.layer2=layer2
        self.layer3=layer3
        self.X_test=X_test
        self.X_train=X_train
        self.Y_test=Y_test
        self.Y_train=Y_train
        #x[0]: relu_size
        #x[1]: weight size layer1
        #x[2]: bias size layer1
        #x[3]: weight size layer2
        #x[4]: bias size layer2
        #x[5]: pruning sparsity
        #x[6]: input size
        #x[7]: int relu size
        
        super().__init__(n_var=8,
                         n_obj=2,
                         n_ieq_constr=0,
                         xl=np.array([3,2,2,2,2,0,2,0]),
                         xu=np.array([8,7,6,7,5,8,4,2]),
                         vtype=int)

    def _evaluate(self, x, out, *args, **kwargs):
        accuracy, weights = bb.blackbox(self.weightsbiases1,self.weightsbiases2, x[0], x[1], x[2], x[3], x[4], x[5], x[6] , x[7],  self.layer1, self.layer2, self.layer3, self.X_test, self.Y_test, self.X_train, self.Y_train)
        f1 = 1- accuracy
        f2 = ar.area(weights,x[6],x[0],x[1],x[3],self.layer1,self.layer2,self.layer3)

        out["F"] = [f1, f2]


In [20]:
layer1=11
layer2=4
layer3=7
problem = MyProblem(wb1,wb2,layer1,layer2,layer3,X_test,X_train,Y_test,Y_train)

In [21]:
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=50,
    sampling=IntegerRandomSampling(),
    crossover=SBX(vtype=float, repair=RoundingRepair()),
    mutation=PM(vtype=float, repair=RoundingRepair()),
    eliminate_duplicates=True
)

In [22]:
from pymoo.termination import get_termination

termination = get_termination("n_gen",50)

In [ ]:
%%time
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)
X = res.X
F = res.F

In [ ]:
print(1-res.F)

In [ ]:
print(res.X)

### plot all the solutions from the final generation

In [ ]:
paretos=res.X
costs=res.F

In [ ]:
from pymoo.visualization.scatter import Scatter
pop=res.pop
vals=pop.get("F")
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(vals, facecolor="none", edgecolor="red")
plot.show()

### plot the pareto solutions from the final generation

In [ ]:
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res.F, facecolor="none", edgecolor="blue")
plot.show()

### define an optimal solution

In [23]:
layer_1=11
layer_2=4
layer_3=7

In [47]:
# import pickle
# with open("paretos_whitewine", "wb") as fp:   #Pickling
#     pickle.dump(tparetos, fp)
# with open("weights_list_true_paretos_whitewine", "wb") as fp:   #Pickling
#     pickle.dump(true_weights_list, fp)
# with open("accuracies_true_paretos_whitewine", "wb") as fp:   #Pickling
#     pickle.dump(accuracy_list, fp)

In [24]:
# with open("true_paretos_whitewine", "rb") as fp:   # Unpickling
#     paretos = pickle.load(fp)
# with open("costs_whitewine", "rb") as fp:   # Unpickling
#     costs = pickle.load(fp)

### load the true pareto, their weights and their accuracies

In [33]:
with open("paretos_whitewine", "rb") as fp:   # Unpickling
    true_paretos = pickle.load(fp)
with open("weights_list_true_paretos_whitewine", "rb") as fp:   # Unpickling
    true_paretos_weights = pickle.load(fp)
with open("accuracies_true_paretos_whitewine", "rb") as fp:   # Unpickling
    true_paretos_accuracies = pickle.load(fp)

### import pareto_verilogs as pv
import pareto_weight_sharing_verilogs as pwsv
import pareto_clustering_verilogs as pcv
import pareto_only_quantization_verilogs as poqv
import only_pruning_verilog as opv
from importlib import reload

In [26]:
import pareto_verilogs as pv
#import pareto_weight_sharing_verilogs as pwsv
import pareto_clustering_verilogs as pcv
#import pareto_only_quantization_verilogs as poqv
import only_pruning_verilog as opv
from importlib import reload
reload(pv)
#reload(pwsv)
reload(pcv)
#reload(poqv)
reload(opv)

<module 'only_pruning_verilog' from '/home/argykokk/Desktop/TC/genetic_whitewine/only_pruning_verilog.py'>

In [ ]:
i=0
layer1=11
layer2=4
layer3=7
epochs=2
lr=0.001
#check the only pruning scenario
opv.generate(i,layer1,layer2,layer3,lr,epochs,X_train,Y_train,X_test,Y_test,wb1,wb2)

In [ ]:
i=0
for solution in paretos:
    relusize_f=solution[0]
    weight_size_f1=solution[1]
    bias_size_f1=solution[2]
    weight_size_f2=solution[3]
    bias_size_f2=solution[4]
    sparsity=solution[5]
    inputsize=solution[6]
    relusize_int=solution[7]
    model=pv.generate(i,relusize_f,weight_size_f1,bias_size_f1,weight_size_f2,bias_size_f2,sparsity,inputsize,relusize_int,layer1,layer2,layer3,lr,epochs,X_train,Y_train,X_test,Y_test,wb1,wb2)
    #pwsv.generate(model,i,relusize_f,weight_size_f1,bias_size_f1,weight_size_f2,bias_size_f2,inputsize,relusize_int,layer1,layer2,layer3,X_train,Y_train,X_test,Y_test)
    #pcv.generate(model,i,relusize_f,weight_size_f1,bias_size_f1,weight_size_f2,bias_size_f2,inputsize,relusize_int,1,0,1,0,layer1,layer2,layer3,X_train,Y_train,X_test,Y_test)
    #poqv.generate(i,relusize_f,weight_size_f1,bias_size_f1,weight_size_f2,bias_size_f2,inputsize,relusize_int,layer1,layer2,layer3,lr,epochs,X_train,Y_train,X_test,Y_test,wb1,wb2)
    i += 1

### only custom sharing

In [30]:
import sharing as sh
reload(sh)
bits_l1=7
bits_int_l1=2
bits_l2=7
bits_int_l2=2


bias_l1=7
bias_l2=7
bias_int_l1=1
bias_int_l2=1
relu_int=2
relu=8
layer1 = 11
layer2 = 4
layer3 = 7

window=6
for i in range(1,layer2+1):
    for j in range(1,layer3+1):
        c1 = i
        c2 = j
        sh.top_sharing(weights1, bits_l1, bits_int_l1, bits_l2, bits_int_l2, bias_l1, bias_int_l1, bias_l2, bias_int_l2, 4, relu, relu_int, X_test, Y_test, layer1, layer2, layer3, c1, c2, window)

/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


46/46 [==============================] - 0s 1ms/step - loss: 1.5851 - accuracy: 0.1918
accuracy = 0.19183672964572906
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


46/46 [==============================] - 0s 987us/step - loss: 1.3814 - accuracy: 0.4544
accuracy = 0.4544217586517334
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

46/46 [==============================] - 0s 1ms/step - loss: 1.1369 - accuracy: 0.5245
accuracy = 0.5244898200035095
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


46/46 [==============================] - 0s 1ms/step - loss: 1.1367 - accuracy: 0.5245
accuracy = 0.5244898200035095
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


46/46 [==============================] - 0s 1ms/step - loss: 1.1367 - accuracy: 0.5245
accuracy = 0.5244898200035095
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


### end of only weight sharing

### start of pareto weight sharing

In [35]:
tparetos=[]

In [40]:
tparetos.append(true_paretos[17])

In [41]:
tparetos

[array([4, 2, 5, 2, 4, 7, 4, 1]),
 array([5, 3, 4, 2, 5, 5, 4, 1]),
 array([4, 5, 5, 5, 3, 4, 4, 1]),
 array([3, 2, 5, 2, 2, 8, 4, 1]),
 array([3, 2, 6, 2, 3, 8, 4, 1])]

In [69]:
true_weights_list=[]
accuracy_list=[]
for sol in tparetos:
    identity=0
    
    import weightsharing as ws
    from importlib import reload
    
    sparsity_val = float(sol[5]/10)
    relusize_f = int(sol[0])
    weight_size_f1 = int(sol[1])
    bias_size_f1 = int(sol[2])
    weight_size_f2 = int(sol[3])
    bias_size_f2 = int(sol[4])
    input_s = int(sol[6])
    int_relu = int(sol[7])
    int_w1=1
    int_b1=0
    int_w2=1
    int_b2=0
    input_size = int(input_s)

    model = Sequential()

    layer1=11
    layer2=4
    layer3=7
    i=99
    epochs=5
    lr=0.001
    X = df.drop('Y', axis = 1).values
    y = df.Y
    le = LabelEncoder()
    print(y)
    y = le.fit_transform(y)
    print(le.classes_)
    y = to_categorical(y, 7)
    X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=42)
    print(X_train.shape)
    # reload(pv)
    # model=pv.generate(i,relusize_f,weight_size_f1,bias_size_f1,weight_size_f2,bias_size_f2,sparsity,inputsize,relusize_int,layer1,layer2,layer3,lr,epochs,X_train,Y_train,X_test,Y_test,wb1,wb2)
    norm = 2**input_s
    sc = MinMaxScaler(feature_range=(0,0.9))
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    for i in range(0,len(X_train)):
        X_train[i] = [int(x*norm)/norm for x in X_train[i]]
    for i in range(0,len(X_test)):
        X_test[i] = [int(x*norm)/norm for x in X_test[i]]


    print("INPUT IS " + str(input_s)+ "NORM IS " + str(norm)+ "RELU IS "+str(relusize_f)+" INT RELU" +str(int_relu)+" WEIGHTS 1 ARE "+str(weight_size_f1)+" BIASES 1 ARE "+str(bias_size_f1)+" WEIGHTS 2 ARE "+str(weight_size_f2)+" BIASES 2 ARE "+str(bias_size_f2)+" SPARSITY : "+str(sparsity_val))

    weight_bias_size=[ [ (weight_size_f1,2), (bias_size_f1,1) ], [ (weight_size_f2,2), (bias_size_f2,1) ] ]
    relu_size=(relusize_f,int_relu)

    model.add(QDense(layer2, input_shape=(layer1,), name='fc1', kernel_quantizer=quantized_bits(weight_bias_size[0][0][0],1,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(weight_bias_size[0][1][0],0,alpha=1),
                    kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)   ))
    model.add(QActivation(activation=quantized_relu(relu_size[0],relu_size[1],use_stochastic_rounding=False), name='relu1'))
    model.add(QDense(layer3, name='output',
                    kernel_quantizer=quantized_bits(weight_bias_size[1][0][0],1,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(weight_bias_size[1][1][0],0,alpha=1),
                    kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
    model.add(Activation(activation='softmax', name='softmax'))


    pruning_params = {"pruning_schedule" : pruning_schedule.ConstantSparsity(sparsity_val, begin_step=300, frequency=100)}
    model = prune.prune_low_magnitude(model, **pruning_params)

    model.layers[0].set_weights(wb1)
    model.layers[2].set_weights(wb2)

    adam = Adam(lr=0.001)

    model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
    callbacks= all_callbacks( outputDir = 'seeds_classification_prune')
    callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
    model.fit(X_train, Y_train, batch_size=1,
                epochs=2,validation_split=0.2, verbose=1, shuffle=True,
                callbacks = callbacks.callbacks);
    model = strip_pruning(model)
    model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
    model_save_quantized_weights(model, "test_weights")


    accuracy=model.evaluate(X_test,Y_test)
    print(model.get_weights())
    print(" ACCURACY IS "+str(accuracy[1]) )


    weights=model.trainable_variables
    weights1=model.trainable_variables
    true_weights_list.append(model.trainable_variables)
    accuracy_list.append(accuracy[1])

0       6
1       6
2       6
3       6
4       6
       ..
4893    6
4894    5
4895    6
4896    7
4897    6
Name: Y, Length: 4898, dtype: int64
[3 4 5 6 7 8 9]
(3428, 11)
INPUT IS 4NORM IS 16RELU IS 4 INT RELU1 WEIGHTS 1 ARE 2 BIASES 1 ARE 5 WEIGHTS 2 ARE 2 BIASES 2 ARE 4 SPARSITY : 0.7


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer LecunUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/2
2742/2742 [==============================] - 6s 2ms/step - loss: 1.4457 - accuracy: 0.4263 - val_loss: 1.3119 - val_accuracy: 0.4883 - lr: 0.0010
Epoch 2/2
2742/2742 [==============================] - 5s 2ms/step - loss: 1.2807 - accuracy: 0.5015 - val_loss: 1.2702 - val_accuracy: 0.5262 - lr: 0.0010
... quantizing model
46/46 [==============================] - 0s 1ms/step - loss: 1.2462 - accuracy: 0.5190
[array([[-1., -1.,  0.,  0.],
       [ 0., -1., -2.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 1.,  1.,  0.,  0.],
       [ 1., -2.,  0.,  0.],
       [-1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.]], dtype=float32), array([ 0.9375,  0.9375,  0.9375, -0.5   ], dtype=float32), array([[ 0.,  0.,  1.,  0.,  0., -2., -2.],
       [-2., -2.,  0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=

/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer LecunUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2742/2742 [==============================] - 6s 2ms/step - loss: 1.3307 - accuracy: 0.4639 - val_loss: 1.1739 - val_accuracy: 0.5073 - lr: 0.0010
Epoch 2/2
2742/2742 [==============================] - 5s 2ms/step - loss: 1.1835 - accuracy: 0.4938 - val_loss: 1.1680 - val_accuracy: 0.5160 - lr: 0.0010
... quantizing model
46/46 [==============================] - 0s 1ms/step - loss: 1.1651 - accuracy: 0.5367
[array([[-1.5, -1.5, -0.5,  0. ],
       [ 1.5, -1. , -2. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [-1.5,  1.5,  0.5,  0. ],
       [ 1.5,  0. ,  1.5,  0. ],
       [ 0. ,  1. ,  0. ,  0. ],
       [ 1.5,  1.5,  0. ,  0. ],
       [ 1.5, -1.5,  0. ,  0. ],
       [-1. , -0.5,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [-1.5,  0.5,  1. ,  0. ]], dtype=float32), array([ 0.875,  0.875,  0.625, -0.5  ], dtype=float32), array([[ 0.,  1.,  1.,  0., -2., -2., -2.],
       [-2., -2.,  0.,  0.,  1.,  1.,  0.],
       [ 0., -1.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.

...

Epoch 1/2


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


2742/2742 [==============================] - 6s 2ms/step - loss: 1.4591 - accuracy: 0.3869 - val_loss: 1.2875 - val_accuracy: 0.4956 - lr: 0.0010
Epoch 2/2
2742/2742 [==============================] - 5s 2ms/step - loss: 1.2928 - accuracy: 0.4781 - val_loss: 1.2610 - val_accuracy: 0.4898 - lr: 0.0010
... quantizing model
46/46 [==============================] - 0s 1ms/step - loss: 1.2635 - accuracy: 0.4714
[array([[-1., -1.,  0.,  0.],
       [ 0., -1., -1.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.]], dtype=float32), array([ 0.9375,  0.9375,  0.9375, -0.5   ], dtype=float32), array([[ 0.,  0.,  1.,  0.,  0., -2., -2.],
       [-2., -2.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32), 

/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer LecunUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/2
2742/2742 [==============================] - 6s 2ms/step - loss: 1.4338 - accuracy: 0.4023 - val_loss: 1.2664 - val_accuracy: 0.5029 - lr: 0.0010
Epoch 2/2
2742/2742 [==============================] - 5s 2ms/step - loss: 1.2861 - accuracy: 0.4905 - val_loss: 1.2571 - val_accuracy: 0.5160 - lr: 0.0010
... quantizing model
46/46 [==============================] - 0s 1ms/step - loss: 1.2574 - accuracy: 0.5095
[array([[-1., -1.,  0.,  0.],
       [ 0., -1., -2.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.]], dtype=float32), array([ 0.96875,  0.96875,  0.96875, -0.5    ], dtype=float32), array([[ 0.,  0.,  1.,  0.,  0., -2., -2.],
       [-2., -2.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]], dt

In [43]:
accuracy_list

[0.5190476179122925,
 0.5367347002029419,
 0.5496598482131958,
 0.4714285731315613,
 0.5095238089561462]

In [70]:
true_weights_list[2]

[<tf.Variable 'fc1/kernel:0' shape=(11, 4) dtype=float32, numpy=
 array([[-1.5  , -1.5  , -0.625,  0.   ],
        [ 1.5  , -1.25 , -2.   ,  0.   ],
        [ 0.25 ,  0.25 ,  0.5  ,  0.   ],
        [-1.625,  1.875,  0.625,  0.   ],
        [ 1.125,  0.   ,  1.125,  0.   ],
        [ 0.   ,  1.   ,  0.   ,  0.   ],
        [ 1.875,  1.875,  0.   ,  0.   ],
        [ 1.875, -1.75 ,  0.   ,  0.   ],
        [-1.125, -0.625, -0.25 ,  0.   ],
        [-0.125,  0.   ,  0.   ,  0.   ],
        [-1.625,  0.125,  1.   ,  0.   ]], dtype=float32)>,
 <tf.Variable 'fc1/bias:0' shape=(4,) dtype=float32, numpy=array([ 0.9375,  0.9375,  0.4375, -0.5   ], dtype=float32)>,
 <tf.Variable 'output/kernel:0' shape=(4, 7) dtype=float32, numpy=
 array([[ 0.75 ,  1.625,  1.875,  0.5  , -2.   , -2.   , -2.   ],
        [-2.   , -2.   , -0.25 ,  0.5  ,  1.75 ,  1.875,  0.   ],
        [-0.875, -0.25 ,  0.   ,  1.875,  0.   , -0.25 ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]],
   

### END OF WEIGHT SHARING OF TRUE PARETO

In [58]:
true_weights_list[2]

[<tf.Variable 'fc1/kernel:0' shape=(11, 4) dtype=float32, numpy=
 array([[-1.5  , -1.5  , -0.625,  0.   ],
        [ 1.5  , -1.25 , -2.   ,  0.   ],
        [ 0.25 ,  0.25 ,  0.5  ,  0.   ],
        [-1.625,  1.875,  0.625,  0.   ],
        [ 1.125,  0.   ,  1.125,  0.   ],
        [ 0.   ,  1.   ,  0.   ,  0.   ],
        [ 1.875,  1.875,  0.   ,  0.   ],
        [ 1.875, -1.75 ,  0.   ,  0.   ],
        [-1.125, -0.625, -0.25 ,  0.   ],
        [-0.125,  0.   ,  0.   ,  0.   ],
        [-1.625,  0.125,  1.   ,  0.   ]], dtype=float32)>,
 <tf.Variable 'fc1/bias:0' shape=(4,) dtype=float32, numpy=array([ 0.9375,  0.9375,  0.4375, -0.5   ], dtype=float32)>,
 <tf.Variable 'output/kernel:0' shape=(4, 7) dtype=float32, numpy=
 array([[ 0.75 ,  1.625,  1.875,  0.5  , -2.   , -2.   , -2.   ],
        [-2.   , -2.   , -0.25 ,  0.5  ,  1.75 ,  1.875,  0.   ],
        [-0.875, -0.25 ,  0.   ,  1.875,  0.   , -0.25 ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]],
   

In [98]:
import pareto_sharing as ps

In [99]:
for w in range(0,len(true_paretos)):
    reload(ps)
    relu= int(true_paretos[w][0])
    bits_l1= int(true_paretos[w][1])
    bias_l1= int(true_paretos[w][2])
    bits_l2=  int(true_paretos[w][3])
    bias_l2=  int(true_paretos[w][4])
    input_s= int(true_paretos[w][6])
    relu_int = int(true_paretos[w][7]) + 1
    bits_int_l1=  2
    bias_int_l1= 1
    bits_int_l2 = 2
    bias_int_l2 = 1
    layer1=11
    layer2=4
    layer3=7

    window=0
    for i in range(1,layer2+1):
        for j in range(1,layer3+1):
            c1 = i
            c2 = j
            ps.top_sharing(w,true_weights_list[w], bits_l1, bits_int_l1, bits_l2, bits_int_l2, bias_l1, bias_int_l1, bias_l2, bias_int_l2, input_s, relu, relu_int, X_test, Y_test, layer1, layer2, layer3, c1, c2, window)
    window=2
    for i in range(1,layer2+1):
        for j in range(1,layer3+1):
            c1 = i
            c2 = j
            ps.top_sharing(w,true_weights_list[w], bits_l1, bits_int_l1, bits_l2, bits_int_l2, bias_l1, bias_int_l1, bias_l2, bias_int_l2, input_s, relu, relu_int, X_test, Y_test, layer1, layer2, layer3, c1, c2, window)  

/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer LecunUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


frozen dataset 0
frozen dataset 1
frozen dataset 2
frozen dataset 3
frozen dataset 4
frozen dataset 5
frozen dataset 6
frozen dataset 7
frozen dataset 8
frozen dataset 9
frozen dataset 10


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


frozen dataset 0
frozen dataset 1
frozen dataset 2
frozen dataset 3


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

46/46 [==============================] - 0s 1ms/step - loss: 1.2574 - accuracy: 0.5095
accuracy = 0.5095238089561462
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer LecunUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


frozen dataset 0
frozen dataset 1
frozen dataset 2
frozen dataset 3
frozen dataset 4
frozen dataset 5
frozen dataset 6
frozen dataset 7
frozen dataset 8
frozen dataset 9
frozen dataset 10
frozen dataset 0
frozen dataset 1
frozen dataset 2
frozen dataset 3


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


46/46 [==============================] - 0s 1ms/step - loss: 1.2574 - accuracy: 0.5095
accuracy = 0.5095238089561462
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


In [62]:
import sharing as sh

w=2
reload(sh)
relu= int(true_paretos[w][0])
bits_l1= int(true_paretos[w][1])
bias_l1= int(true_paretos[w][2])
bits_l2=  int(true_paretos[w][3])
bias_l2=  int(true_paretos[w][4])
input_s= int(true_paretos[w][6])
relu_int = int(true_paretos[w][7]) + 1
bits_int_l1=  2
bias_int_l1= 1
bits_int_l2 = 2
bias_int_l2 = 1
layer1=11
layer2=4
layer3=7

window=0
for i in range(1,layer2+1):
    for j in range(1,layer3+1):
        c1 = i
        c2 = j
        sh.top_sharing(true_weights_list[w], bits_l1, bits_int_l1, bits_l2, bits_int_l2, bias_l1, bias_int_l1, bias_l2, bias_int_l2, input_s, relu, relu_int, X_test, Y_test, layer1, layer2, layer3, c1, c2, window)


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


46/46 [==============================] - 0s 1ms/step - loss: 1.4497 - accuracy: 0.2789
accuracy = 0.2789115607738495
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


46/46 [==============================] - 0s 1ms/step - loss: 1.3933 - accuracy: 0.4129
accuracy = 0.41292518377304077
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


...

46/46 [==============================] - 1s 1ms/step - loss: 1.3575 - accuracy: 0.4741
accuracy = 0.4741496741771698
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


/home/argykokk/Desktop/TC/genetic_whitewine/sharing.py:167: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))
/home/argykokk/Desktop/TC/genetic_whitewine/sharing.py:167: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))
/home/argykokk/Desktop/TC/genetic_whitewine/sharing.py:167: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecat

46/46 [==============================] - 0s 1ms/step - loss: 1.3575 - accuracy: 0.4741
accuracy = 0.4741496741771698
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


/home/argykokk/Desktop/TC/genetic_whitewine/sharing.py:167: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))
/home/argykokk/Desktop/TC/genetic_whitewine/sharing.py:167: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))
/home/argykokk/Desktop/TC/genetic_whitewine/sharing.py:167: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))
/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecat

46/46 [==============================] - 0s 1ms/step - loss: 1.3575 - accuracy: 0.4741
accuracy = 0.4741496741771698
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
copy top.v, sim.Xtest, and sim.Ytest to the synopsys project 


In [95]:
w=2
import collections
from importlib import reload

from mult_area import mult_area
import write_mlp_mergemult_ps as wv1
from sklearn.cluster import KMeans
import copy

relu= int(true_paretos[w][0])
bits_l1= int(true_paretos[w][1])
bias_l1= int(true_paretos[w][2])
bits_l2=  int(true_paretos[w][3])
bias_l2=  int(true_paretos[w][4])
input_s= int(true_paretos[w][6])
relu_int = int(true_paretos[w][7]) + 1
bits_int_l1=  2
bias_int_l1= 1
bits_int_l2 = 2
bias_int_l2 = 1
layer1=11
layer2=4
layer3=7

window=0

relusize_f= relu
weight_size_f1= bits_l1
bias_size_f1= bias_l1
weight_size_f2= bits_l2
bias_size_f2= bias_l2
#input_s= inputsize
int_relu = relu_int
int_w1= bits_int_l1
int_b1= bias_int_l1
int_w2= bits_int_l2
int_b2= bias_int_l2
half_window = int(window / 2)


# create a model with the selected bitwidths and find the zero cost weights (or the duplicates)
model1 = Sequential()
model1.add(QDense(layer2, input_shape=(layer1,), name='fc1', kernel_quantizer=quantized_bits(bits_l1,bits_int_l1-1,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(bias_l1,bias_int_l1-1,0,alpha=1),kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)   ))
model1.add(QActivation(activation=quantized_relu(relu,relu_int-1,use_stochastic_rounding=False), name='relu1'))
model1.add(QDense(layer3, name='output',
          kernel_quantizer=quantized_bits(bits_l2,bits_int_l2-1,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(bias_l2,bias_int_l2-1,alpha=1),
          kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model1.add(Activation(activation='softmax', name='softmax'))
model1.set_weights(true_weights_list[2])

adam = Adam(lr=0.00001)
            #callbacks= all_callbacks( outputDir = 'callbacks')
model1.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
new_accuracy=model1.evaluate(X_test,Y_test)
print("B accuracy = "+str(new_accuracy[1]))

# for the two layers
for node in range(0,1):
    GoldenList1=[]
    GoldenList2=[]
    for i in range(0,bits_l1):
        GoldenList1.append( 2**i/(2**(bits_l1-bits_int_l1) ))
        GoldenList1.append(- 2**i/(2**(bits_l1-bits_int_l1) ))
    for i in range(0,bits_l2):
        GoldenList2.append(2**i/(2**(bits_l2-bits_int_l2) ))
        GoldenList2.append(-2**i/(2**(bits_l2-bits_int_l2) ))
    GoldenList1.append(0)
    GoldenList2.append(0)


    # We will create "layer1" datasets for the first set of weights and "layer2" datasets for the second set
    # Each one of the first set datasets will contain "layer2" weights and each one of the second set of datasets will contain "layer3" weights
    for ii in range(0,1):

        target_layer = ii

        # make an intialization depending if we aim the first or the second sets
        if target_layer==0:
            GoldenList=GoldenList1
            inputs=layer1
            tvars=model1.trainable_variables[0]
            bits=bits_l1
            bits_int=bits_int_l1
            i_bits=input_s
            desired_clusters= 4
        else:
            GoldenList=GoldenList2
            inputs=layer2
            tvars=model1.trainable_variables[2]
            bits=bits_l2
            target_layer=2
            bits_int=bits_int_l2
            i_bits=relu
            desired_clusters= 7

        # extract the datasets in a list format
        l=[]
        datasets=[]
        for i in range(0,inputs):
            l.append(tf.get_static_value(tvars[i]))

        for i in range(0,inputs):
            ls=[]
            for i in l[i]:
                ls.append(float(i))
            datasets.append(ls)

        #create the datasets for the clustering
        cluster_datasets=[]
        positions=[]
        frozen_datasets= [0] * inputs
        for i in range(0,inputs):
            ls_new=[]
            pos=[]
            index=0
            for j in datasets[i]:
                if j not in GoldenList:
                    ls_new.append(j)
                    pos.append(index)
                index = index + 1
            if len(ls_new):
                cluster_datasets.append(ls_new)
                positions.append(pos)
            else:
                frozen_datasets[i]=1
                #place holder
                cluster_datasets.append([0])
                positions.append([0])


        #do clustering
        for dataset in cluster_datasets:

            #re-define the number of clusters
            if desired_clusters > len(dataset):
                num_clusters = len(dataset)
            else:
                num_clusters = desired_clusters
            array = np.array(dataset)
            kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))
            centroids=kmeans.cluster_centers_.reshape(-1).tolist()

            # round the centroids up to 6 decimals accuracy
            for i in range(0,len(centroids)):
                centroids[i]=round(centroids[i],6)
                # calculate the estimated area overhead of this centroid
                #save the sign
                qcentroid = int(centroids[i]*(2 ** (bits-bits_int)))
                final_centroid = qcentroid
                multarea = mult_area[int(i_bits)][abs(int(qcentroid))]
                #print("here")

                # now for half window to the right if (< 127) and half window to the left if (> 0 ) evaluate the neighbor points
                for w in range(1,half_window+1):
                    new_centroid = w + qcentroid
                    if new_centroid < 127:
                        new_multarea = mult_area[int(i_bits)][abs(int(new_centroid))]
                        if new_multarea < multarea:
                            multarea = new_multarea
                            final_centroid = new_centroid

                for w in range(1,half_window+1):
                    new_centroid = qcentroid - w
                    if new_centroid > 0:
                        new_multarea = mult_area[int(i_bits)][abs(int(new_centroid))]
                        if new_multarea < multarea:
                            multarea = new_multarea
                            final_centroid = new_centroid
                fixed_point_centroid = final_centroid/(2 ** (bits-bits_int))
                #print("FOUND ANOTHER CENTROID"+str(final_centroid)+" AKA "+str(fixed_point_centroid)+" INSTEAD OF "+str(qcentroid)+" AKA "+str(centroids[i]))
                centroids[i] = fixed_point_centroid
            labels=kmeans.labels_.tolist()
            index = 0
            for i in range(0,len(dataset)):
                dataset[i] = centroids[labels[i]]
                dataset[i] = int(dataset[i]*(2 ** (bits-bits_int)))
                dataset[i] = dataset[i]/(2 ** (bits-bits_int))


        #restore weights
        restored_weights= copy.deepcopy(datasets)
        for i in range(0,inputs):
            if frozen_datasets[i]==1:
                print("frozen dataset "+str(i))
            else:
                index = 0
                for j in positions[i]:
                    restored_weights[i][j] = cluster_datasets[i][index]
                    index = index + 1

        model1.trainable_variables[target_layer].assign(restored_weights)
        adam = Adam(lr=0.00001)
            #callbacks= all_callbacks( outputDir = 'callbacks')
model1.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
new_accuracy=model1.evaluate(X_test,Y_test)
print("accuracy = "+str(new_accuracy[1]))


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


46/46 [==============================] - 0s 2ms/step - loss: 1.1369 - accuracy: 0.5497
B accuracy = 0.5496598482131958
dataset 2 [0.25, 0.25, 0.5, 0.0] is frozen hmm
dataset 5 [0.0, 1.0, 0.0, 0.0] is frozen hmm
dataset 9 [-0.125, 0.0, 0.0, 0.0] is frozen hmm


/tmp/ipykernel_9611/3610571300.py:141: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))
/tmp/ipykernel_9611/3610571300.py:141: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))
/tmp/ipykernel_9611/3610571300.py:141: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=num_clusters,random_state=0, n_init=10).fit(array.reshape(-1,1))


frozen dataset 2
frozen dataset 5
frozen dataset 9


/home/argykokk/miniconda3/envs/hls4ml-tutorial/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


46/46 [==============================] - 0s 1ms/step - loss: 1.1369 - accuracy: 0.5497
accuracy = 0.5496598482131958


In [86]:
fdatasets= [0]*inputs

In [89]:
fdatasets

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [76]:
datasets

[[-1.5, -1.5, -0.625, 0.0],
 [1.5, -1.25, -2.0, 0.0],
 [0.25, 0.25, 0.5, 0.0],
 [-1.625, 1.875, 0.625, 0.0],
 [1.125, 0.0, 1.125, 0.0],
 [0.0, 1.0, 0.0, 0.0],
 [1.875, 1.875, 0.0, 0.0],
 [1.875, -1.75, 0.0, 0.0],
 [-1.125, -0.625, -0.25, 0.0],
 [-0.125, 0.0, 0.0, 0.0],
 [-1.625, 0.125, 1.0, 0.0]]

In [80]:
positions

[[0, 1, 2], [0, 1], [0, 1, 2], [0, 2], [0, 1], [0, 1], [0, 1], [0]]

In [77]:
cluster_datasets

[[-1.5, -1.5, -0.625],
 [1.5, -1.25],
 [-1.625, 1.875, 0.625],
 [1.125, 1.125],
 [1.875, 1.875],
 [1.875, -1.75],
 [-1.125, -0.625],
 [-1.625]]

In [96]:
model1.get_weights()

[array([[-1.5  , -1.5  , -0.625,  0.   ],
        [ 1.5  , -1.25 , -2.   ,  0.   ],
        [ 0.25 ,  0.25 ,  0.5  ,  0.   ],
        [-1.625,  1.875,  0.625,  0.   ],
        [ 1.125,  0.   ,  1.125,  0.   ],
        [ 0.   ,  1.   ,  0.   ,  0.   ],
        [ 1.875,  1.875,  0.   ,  0.   ],
        [ 1.875, -1.75 ,  0.   ,  0.   ],
        [-1.125, -0.625, -0.25 ,  0.   ],
        [-0.125,  0.   ,  0.   ,  0.   ],
        [-1.625,  0.125,  1.   ,  0.   ]], dtype=float32),
 array([ 0.9375,  0.9375,  0.4375, -0.5   ], dtype=float32),
 array([[ 0.75 ,  1.625,  1.875,  0.5  , -2.   , -2.   , -2.   ],
        [-2.   , -2.   , -0.25 ,  0.5  ,  1.75 ,  1.875,  0.   ],
        [-0.875, -0.25 ,  0.   ,  1.875,  0.   , -0.25 ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]],
       dtype=float32),
 array([-1.  ,  0.25,  0.75,  0.75,  0.75, -0.75, -1.  ], dtype=float32)]

In [68]:
true_weights_list[2]

[<tf.Variable 'fc1/kernel:0' shape=(11, 4) dtype=float32, numpy=
 array([[-1.5  , -1.5  , -0.625,  0.   ],
        [ 1.5  , -1.25 , -2.   ,  0.   ],
        [ 0.25 ,  0.25 ,  0.5  ,  0.   ],
        [-1.625,  1.875,  0.625,  0.   ],
        [ 1.125,  0.   ,  1.125,  0.   ],
        [ 0.   ,  1.   ,  0.   ,  0.   ],
        [ 1.875,  1.875,  0.   ,  0.   ],
        [ 1.875, -1.75 ,  0.   ,  0.   ],
        [-1.125, -0.625, -0.25 ,  0.   ],
        [-0.125,  0.   ,  0.   ,  0.   ],
        [-1.625,  0.125,  1.   ,  0.   ]], dtype=float32)>,
 <tf.Variable 'fc1/bias:0' shape=(4,) dtype=float32, numpy=array([ 0.9375,  0.9375,  0.4375, -0.5   ], dtype=float32)>,
 <tf.Variable 'output/kernel:0' shape=(4, 7) dtype=float32, numpy=
 array([[ 0.75 ,  1.625,  1.875,  0.5  , -2.   , -2.   , -2.   ],
        [-2.   , -2.   , -0.25 ,  0.5  ,  1.75 ,  1.875,  0.   ],
        [-0.875, -0.25 ,  0.   ,  1.875,  0.   , -0.25 ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]],
   

In [59]:
true_paretos

[array([4, 2, 5, 2, 4, 7, 4, 1]),
 array([5, 3, 4, 2, 5, 5, 4, 1]),
 array([4, 5, 5, 5, 3, 4, 4, 1]),
 array([3, 2, 5, 2, 2, 8, 4, 1]),
 array([3, 2, 6, 2, 3, 8, 4, 1])]

In [55]:
accuracy_list

[0.5190476179122925,
 0.5367347002029419,
 0.5496598482131958,
 0.4714285731315613,
 0.5095238089561462]